# Arcane Python syntax

Python has a syntax with a few features that are quite unique.

**General advice:** don't use any of this unless you feel comfortable with it, since mistakes can lead to bugs that are hard to track down.

## Interval checking

In Python an expression such as `a < x <= b` is legal and well-defined.

In [1]:
a, b = 1, 3
for x in range(-2, 5):
    if a < x <= b:
        print(f'{a} < {x} <= {b}: True')
    else:
        print(f'{a} < {x} <= {b}: False')

1 < -2 <= 3: False
1 < -1 <= 3: False
1 < 0 <= 3: False
1 < 1 <= 3: False
1 < 2 <= 3: True
1 < 3 <= 3: True
1 < 4 <= 3: False


The code above can be simplified to:

In [3]:
a, b = 1, 3
for x in range(-2, 5):
    print(f'{a} < {x:2d} <= {b}: {a < x <= b}')

1 < -2 <= 3: False
1 < -1 <= 3: False
1 <  0 <= 3: False
1 <  1 <= 3: False
1 <  2 <= 3: True
1 <  3 <= 3: True
1 <  4 <= 3: False


## Multiple equality, inequality

Along the same lines, `a == b == x` is also legal and well-defined.

In [4]:
for a in range(3):
    for b in range(3):
        print(f'{a} == {b} == 1: {a == b == 1}')

0 == 0 == 1: False
0 == 1 == 1: False
0 == 2 == 1: False
1 == 0 == 1: False
1 == 1 == 1: True
1 == 2 == 1: False
2 == 0 == 1: False
2 == 1 == 1: False
2 == 2 == 1: False


Although `a != b != x` is legal as well, it may, at least at first sight, not behave as expected.

In [5]:
for a in range(3):
    for b in range(3):
        print(f'{a} != {b} != 1: {a != b != 1}')

0 != 0 != 1: False
0 != 1 != 1: False
0 != 2 != 1: True
1 != 0 != 1: True
1 != 1 != 1: False
1 != 2 != 1: True
2 != 0 != 1: True
2 != 1 != 1: False
2 != 2 != 1: False


From the above, it is clear that `a != b != x` translates to `a != b and b != c`, which is true when `a == c and a != b`.  From a mathematical point of view, bear in mind that `==` is transitive, while `!=` is not.

## Iteration with `else`

Iteration statements in Python, i.e., `for` and `while` can have an `else` block.  The latter is executed when the iteration statement terminates normally, i.e., not by a `break` statement.

In [6]:
def illustrate_for_else(x):
    for i in range(10):
        if i == x:
            print('break')
            break
    else:
        print('no break')

In [7]:
illustrate_for_else(12)

no break


In [8]:
illustrate_for_else(5)

break


Although naming this syntactic construct `else` feels awkward, it is quite useful, since it is a syntactic shortcut for the following reasonably common construct.

In [12]:
def illustrate_for_bool(x):
    completed_succesfully = True
    for i in range(10):
        if i == x:
            print('break')
            completed_succesfully = False
            break
    if completed_succesfully:
        print('no break')

In [13]:
illustrate_for_bool(12)

no break


In [14]:
illustrate_for_bool(5)

break


The execution of `continue` has no influence on this.

In [10]:
for i in range(5):
    if i > -1:
        continue
    print(f'did something for {i}')
else:
    print('completed normally')

completed normally


The `while` statement can have an `else` with the same semantics.

## Logical shortcircuits

Boolean operators can be used directly as control statements.  This is familiar to experience Bash programmers, but leads to code that is somewhat hard to understand.

In [1]:
import sys

In [5]:
output_file = None
fh = output_file or sys.stdout
print('hello', file=fh)

hello


If the first operand to `or` is `False`, the value of the expression will be that of the second operand. The value `None` converts to Boolean `False`, hence the behavior above. However, if the first operand can be converted to `True`, the value of the expression is that of the first operand, a file handle in the example below.

In [14]:
output_file = open('remove_me.txt', 'w')
fh = output_file or sys.stdout
print('hello', file=fh)

In [15]:
!cat remove_me.txt

hello


The semantics of `and` expressions is similar.  If the first operand converts to `True`, the expression will have the value of the second operand. If the first operand converts to `False`, that operand will be the value of the expression.

In [2]:
a_list = []
b_list = [3, 5, 7]
my_list = a_list and b_list
print(my_list)

[]


In [3]:
a_list = [3, 5, 7]
b_list = [3, 5, 7, 9]
my_list = a_list and b_list
print(my_list)

[3, 5, 7, 9]
